# <center>Quantum Operations & Measurements</center>

In [1]:
import random
import matplotlib.pyplot as plt
from pprint import pprint 

from pennylane import numpy as np
import pennylane as qml
from pennylane.operation import Tensor

#pip install pennylane
#https://pennylane.readthedocs.io/en/stable/introduction/templates.html

In [2]:
dev1 = qml.device("default.qubit", wires=2) #wires = # of qubits/qumodes

In [3]:
@qml.qnode(dev1)
def circuit0(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    return qml.expval(qml.PauliZ(0)) #0 or 1: Z, + or -: X, +i or -i: Y

## Qubit Gates

In [4]:
dev2 = qml.device("default.qubit", wires=4)

In [5]:
pparams = [0.5, 0.3]
n_qubits = 4
paulis = []
for i in range(n_qubits):
    paulis.append(qml.PauliZ(i))

@qml.qnode(dev2)
def circuitGates(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.Rot(0, np.pi/2, 0, wires=2) #X, Y, Z
    qml.CNOT(wires=[0, 1])
    qml.SWAP(wires=[0, 3])
    qml.CSWAP(wires=[1, 2, 3])
    qml.QFT(wires=[0, 1, 2])
    qml.Toffoli(wires=[1, 2, 3])
    qml.MultiControlledX([0, 2, 3], 1)
    qml.U1(np.pi/6, wires=3)
    qml.U2(np.pi/2, -np.pi/5, wires=2)
    qml.U3(-11*np.pi/12, 4*np.pi/5, np.pi, wires=1)
    return qml.expval(Tensor(*paulis))
#Regular Qubit Gates
#Quantum Trapped Ion Qubit Gates

In [6]:
circuitGates(pparams)

tensor(0.529439, requires_grad=True)

## Qubit State Preparation

In [21]:
#Basis States
pparams = [0, 1, 1, 1] #n params for n qubits
    
@qml.qnode(dev2)
def circuitBasis(params):
    qml.BasisState(params, wires=[0, 1, 2, 3])
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

In [22]:
circuitBasis(pparams)

tensor([ 1., -1., -1., -1.], requires_grad=True)

In [23]:
#2^n params, for n qubits

#State Vectors
@qml.qnode(dev2)
def circuitState(params):
    qml.QubitStateVector(params, wires=[0, 1, 2, 3])
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

In [24]:
pparams = [0 for i in range(2**n_qubits)]
pparams[5] = 1
print(pparams)
circuitState(pparams)

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


tensor([ 1., -1.,  1., -1.], requires_grad=True)

## Noisy Channels

In [41]:
#Not all Quantum Computers are perfect, we have errors!
#Simulate Errors, we use quantum Channels
shots=random.randint(1, 50)
dev3 = qml.device("default.mixed", wires=4, shots=shots)
@qml.qnode(dev3)
def circuitNoise(params):
    qml.RX(params[0], wires=3)
    qml.RY(params[1], wires=1)
    qml.AmplitudeDamping(0.1, wires=0)
    qml.PhaseDamping(0.1, wires=1) #Phase of \ket{1}
    qml.BitFlip(0.5, wires=2) #flip 0 to 1, or 1 to 0 if occurs
    qml.PhaseFlip(0.5, wires=3) #1 to -1, or -1 to 1
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

In [42]:
pparams=[0, 0]
for i in range(10):
    print(circuitNoise(pparams))

[1.         1.         0.18918919 1.        ]
[1.         1.         0.08108108 1.        ]
[1.         1.         0.08108108 1.        ]
[1.         1.         0.13513514 1.        ]
[ 1.          1.         -0.02702703  1.        ]
[1.         1.         0.24324324 1.        ]
[ 1.          1.         -0.02702703  1.        ]
[ 1.          1.         -0.02702703  1.        ]
[ 1.          1.         -0.13513514  1.        ]
[ 1.          1.         -0.18918919  1.        ]


## Qubit Observables

In [43]:
@qml.qnode(dev2)
def circuitObs1(params):
    qml.RX(params[0], wires=3)
    qml.RY(params[1], wires=1)
    return qml.expval(qml.PauliX(0) @ qml.PauliY(1) @ qml.PauliZ(2))

In [44]:
@qml.qnode(dev2)
def circuitObs2(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    return qml.expval(qml.Hadamard(0))

In [45]:
pparams=[0, 0]
circuitObs1(pparams)

tensor(0., requires_grad=True)

In [48]:
pparams=[random.randint(0,2)*np.pi/2, 0]
circuitObs2(pparams)

tensor(-0.70710678, requires_grad=True)

## Continuous Variable Gates

In [49]:
n_qumodes = 4 #Quantum Optical Devices
dev4 = qml.device("default.gaussian", wires=n_qumodes)
@qml.qnode(dev4)
def circuitCVGates():
    qml.Displacement(2, np.pi/4, wires=0)
    qml.Squeezing(0.3, -3*np.pi/2, wires=1)
    qml.Beamsplitter(7*np.pi/8, -2*np.pi/5, wires=range(2, 4))
    return qml.expval(qml.NumberOperator(wires=2))

In [50]:
circuitCVGates()

tensor(0., requires_grad=False)

## Continuous Variable Preparation

In [51]:
n_qumodes = 4
dev5 = qml.device("strawberryfields.fock", cutoff_dim=6, wires=n_qumodes)


@qml.qnode(dev5)
def circuitCVPrep():
    qml.FockState(3, wires=2)
    qml.Beamsplitter(np.pi/4, np.pi/2, wires=range(2, 4))
    qml.DisplacedSqueezedState(0.4, np.pi/3, 3, -9*np.pi/5, wires=0)
    qml.Beamsplitter(np.pi/4, np.pi/2, wires=[0, 3])
    return [qml.expval(qml.NumberOperator(wires=2)), qml.expval(qml.NumberOperator(wires=0))]



In [52]:
circuitCVPrep()

tensor([0.2845701 , 0.25854825], requires_grad=False)

## Continuous Variable Observables

In [67]:
@qml.qnode(dev5)
def circuitCVObs1():
    qml.FockState(1, wires=2)
    qml.Beamsplitter(np.pi/4, np.pi/2, wires=range(2, 4))
    qml.DisplacedSqueezedState(0.4, np.pi/3, 3, -9*np.pi/5, wires=0)
    qml.Beamsplitter(np.pi/4, np.pi/2, wires=[0, 3])
    return qml.expval(qml.X(wires=2))

@qml.qnode(dev5)
def circuitCVObs2():
    qml.FockState(1, wires=2)
    qml.Beamsplitter(np.pi/4, np.pi/2, wires=range(2, 4))
    qml.DisplacedSqueezedState(0.4, np.pi/3, 3, -9*np.pi/5, wires=0)
    qml.Beamsplitter(np.pi/4, np.pi/2, wires=[0, 3])
    return qml.expval(qml.P(wires=2))

@qml.qnode(dev5)
def circuitCVObs3():
    qml.FockState(2, wires=0)
    qml.FockState(2, wires=1)
    qml.Beamsplitter(np.pi/5, 0, wires=range(2))
    return qml.expval(qml.FockStateProjector([1, 3], wires=[0, 1]))


In [68]:
#print(circuitCVObs1())
print()
#print(circuitCVObs2())
print()
print(circuitCVObs3())



0.1295593135546973


## Quantum Measurements

In [70]:
dev6 = qml.device("default.qubit", wires=4, shots=10)

@qml.qnode(dev6)
def circuitMeasure1():
    qml.Hadamard(wires=3)
    qml.CNOT(wires=[0, 1])
    qml.SWAP(wires=[0, 3])
    return qml.sample(qml.PauliZ(0))

@qml.qnode(dev6)
def circuitMeasure2():
    qml.Hadamard(wires=3)
    qml.CNOT(wires=[0, 1])
    qml.SWAP(wires=[0, 3])
    return qml.var(qml.PauliZ(0))

@qml.qnode(dev6)
def circuitMeasure3():
    qml.Hadamard(wires=3)
    qml.CNOT(wires=[0, 1])
    qml.SWAP(wires=[0, 3])
    return qml.state()

In [71]:
print(circuitMeasure1())
print()
print(circuitMeasure2())
print()
print(circuitMeasure3())

[-1 -1  1 -1 -1  1 -1  1  1 -1]

0.96

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j]
